In [ ]:
pip install ../

In [11]:
import itertools
import numpy as np
import sys
from io import StringIO
from sccarlinpy.align import  BiasedScoringMatrix, LocalAlignment

In [12]:
# Set up the target reference sequence and primer:
primer_seq = 'CGGATTAACGTGTAAGCGGC'
region_offset=2861
ref = "gcttctgctcggattaaCgtgtaagcggcGCTAGCcgccggactgcacgacagtcgacgatggagtcgacacgactcgcgcatacgatggagtcgactacagtcgctacgacgatggagtcgcgagcgctatgagcgactatggagtcgatacgatacgcgcacgctatggagtcgagagcgcgctcgtcgactatggagtcgcgactgtacgcacacgcgatggagtcgatagtatgcgtacacgcgatggagtcgagtcgagacgctgacgatatggagtcgatacgtagcacgcagacgatgggagctagaattctaactagagctcgctgatcagccCCCGGGtcgactgtgccttctagttgccagccatctgttgtttgcccctcccccgtgccttccttgaccctggaaggtgccactcccactgtcctttcctaataaaatgaggaaattgcatcgcattgtctgagtaggtgtcattctattctggggggtggggtggggcaggacagcaagggggaggattgggaagagaatagcaggcatgctggggaAGATCTtaagctgcaataaacaagttaacaacaacaattgcatt".upper()
ref_name = 'Tigre_d2egfp_carlin_2e12_1f11'
###


primer_offset = ref.index(primer_seq)
scoring = BiasedScoringMatrix(20, -15, biased_reference_positions=set(list(range(primer_offset,primer_offset+len(primer_seq)+1))), 
                              biased_match_score= 80, 
                              biased_query_positions=set(list(range(0,len(primer_seq)+1))))
sw = LocalAlignment(scoring, full_query=True, prefer_gap_runs=True, gap_penalty=-50, gap_extension_penalty=-1 )  

In [4]:
import glob
from multiprocessing import Pool
from collections import defaultdict, Counter
from sccarlinpy import add_to_readset

longreadset = Counter()
shortreadset = Counter()

sequences_per_cell = defaultdict(lambda: defaultdict(Counter)) # library -> cell -> reads [n]


for path in glob.glob('../data/long/*.scstats.tsv'):
    add_to_readset(path, longreadset, sequences_per_cell)    

for path in glob.glob('../data/short/*.scstats.tsv'):
    add_to_readset(path, shortreadset, sequences_per_cell)

In [5]:
# Perform alignments
from sccarlinpy import get_alignment
from tqdm import tqdm

def _get_alignment(read):
    # Requires globals : ref, sw and ref_name
    return get_alignment(read, ref, sw, ref_name)

aln_chunksize = 100
mincount = 2

read_alignments_long = {}
with Pool() as workers:
    for read, alignment in tqdm(workers.imap(_get_alignment, 
                                        (read for read,count in longreadset.items() if count>=mincount), chunksize=aln_chunksize)
                               ):
        read_alignments_long[read] = alignment
    
read_alignments_short = {}
with Pool() as workers:
    for read, alignment in tqdm(workers.imap(_get_alignment, (read for read,count in shortreadset.items() if count>=mincount), chunksize=aln_chunksize)):
        read_alignments_short[read] = alignment

68880it [03:15, 352.69it/s] 
316610it [10:15, 514.31it/s] 


In [6]:
from singlecellmultiomics.utils import hamming_distance

safe_short_to_longer = dict() 
unsafe_short_to_longer = dict()
short_missing_in_longer = set()

def read_to_safe_unsafe(args):
    short_read, short_alignment = args
    unsafe = set()
    safe = set()
    is_safe = True
    for longer_read, long_alignment in read_alignments_long.items():
        prefix = longer_read[:len(short_read)]
        d = hamming_distance(prefix, short_read)
        if d<=1:
            if long_alignment==short_alignment:
                safe.add(longer_read)
                # The longer read has the same alignment as the shorter read, likely no ambiguity
            else:
                unsafe.add(longer_read)
                is_safe = False # There are multiple long read alignments which fit this shorter read
    return short_read, safe, unsafe, is_safe



with Pool() as workers:
    for i, (shortread, safe, unsafe, is_safe) in enumerate(workers.imap_unordered(read_to_safe_unsafe, read_alignments_short.items(), chunksize=100)):
        if i%1000==0:
            print(f'{i} {len(read_alignments_short)}  ',  end='\r')
        if is_safe and len(safe)>0:
            safe_short_to_longer[shortread] = safe
        elif not is_safe:
            unsafe_short_to_longer[shortread] = unsafe.union(safe)
        else:
            short_missing_in_longer.add(shortread) # This sequence is not available

In [7]:
print('Number of short sequences:', len(read_alignments_short))
print('Number of long sequences:', len(read_alignments_long))
print()
print('Number of short sequences safely convertable to long:', len(safe_short_to_longer))
print('Number of short sequences which are ambigous convertable to long:', len(unsafe_short_to_longer))
print('Number of short sequences which are not found in the long read set:', len(short_missing_in_longer))

Number of short sequences: 316610
Number of long sequences: 68880

Number of short sequences safely convertable to long: 114591
Number of short sequences which are ambigous convertable to long: 43775
Number of short sequences which are not found in the long read set: 158244


In [8]:
sam_header = """@SQ	SN:1	LN:195154279
@SQ	SN:10	LN:130530862
@SQ	SN:11	LN:121973369
@SQ	SN:12	LN:120092757
@SQ	SN:13	LN:120883175
@SQ	SN:14	LN:125139656
@SQ	SN:15	LN:104073951
@SQ	SN:16	LN:98008968
@SQ	SN:17	LN:95294699
@SQ	SN:18	LN:90720763
@SQ	SN:19	LN:61420004
@SQ	SN:2	LN:181755017
@SQ	SN:3	LN:159745316
@SQ	SN:4	LN:156860686
@SQ	SN:5	LN:151758149
@SQ	SN:6	LN:149588044
@SQ	SN:7	LN:144995196
@SQ	SN:8	LN:130127694
@SQ	SN:9	LN:124359700
@SQ	SN:MT	LN:16299
@SQ	SN:X	LN:169476592
@SQ	SN:Y	LN:91455967
@SQ	SN:JH584299.1	LN:953012
@SQ	SN:GL456233.2	LN:559103
@SQ	SN:JH584301.1	LN:259875
@SQ	SN:GL456211.1	LN:241735
@SQ	SN:GL456221.1	LN:206961
@SQ	SN:JH584297.1	LN:205776
@SQ	SN:JH584296.1	LN:199368
@SQ	SN:GL456354.1	LN:195993
@SQ	SN:JH584298.1	LN:184189
@SQ	SN:JH584300.1	LN:182347
@SQ	SN:GL456219.1	LN:175968
@SQ	SN:GL456210.1	LN:169725
@SQ	SN:JH584303.1	LN:158099
@SQ	SN:JH584302.1	LN:155838
@SQ	SN:GL456212.1	LN:153618
@SQ	SN:JH584304.1	LN:114452
@SQ	SN:GL456379.1	LN:72385
@SQ	SN:GL456366.1	LN:47073
@SQ	SN:GL456367.1	LN:42057
@SQ	SN:GL456239.1	LN:40056
@SQ	SN:GL456383.1	LN:38659
@SQ	SN:GL456385.1	LN:35240
@SQ	SN:GL456360.1	LN:31704
@SQ	SN:GL456378.1	LN:31602
@SQ	SN:MU069435.1	LN:31129
@SQ	SN:GL456389.1	LN:28772
@SQ	SN:GL456372.1	LN:28664
@SQ	SN:GL456370.1	LN:26764
@SQ	SN:GL456381.1	LN:25871
@SQ	SN:GL456387.1	LN:24685
@SQ	SN:GL456390.1	LN:24668
@SQ	SN:GL456394.1	LN:24323
@SQ	SN:GL456392.1	LN:23629
@SQ	SN:GL456382.1	LN:23158
@SQ	SN:GL456359.1	LN:22974
@SQ	SN:GL456396.1	LN:21240
@SQ	SN:GL456368.1	LN:20208
@SQ	SN:MU069434.1	LN:8412
@SQ	SN:JH584295.1	LN:1976
@SQ	SN:Tigre_d2egfp_carlin_2e12_1f11	LN:5346\n"""

In [14]:
# Write all data (long form):
import gzip
from singlecellmultiomics.utils import pool_wrapper
from tqdm.notebook import tqdm
import os 
alignment_commands = {}

def alignment_wrapper(query_name, query_sequence, cell, scar ):
    # Depends on region_offet, ref,
    aln = sw.align(ref, query_sequence, query_name=query_name, ref_name=ref_name, q_qual=None)
    q_name, flag, r_name,  start, mapq, cigar, _, __, tlen, seq, q_qual = aln.dump_sam(region_offset)
    q_qual = 'A' * len(seq)                    
    return '\t'.join([str(x) for x in [q_name, flag, r_name,  start, mapq, cigar, _, __, tlen, seq, q_qual, f'SC:Z:{scar}', f'SM:Z:{cell}']])+ '\n'

def align_sequences_to_sam(sam_out_path, commands):
    with open(sam_out_path,'w') as samout:
        samout.write(sam_header)
        with Pool() as workers:
            for sam_line in tqdm( workers.imap(pool_wrapper, ( (alignment_wrapper, kwargs) for kwargs in commands)), total=len(commands) ):
                samout.write( sam_line )                   
    bam_path = sam_out_path.replace('.sam', '.bam')
    os.system(f"samtools sort {sam_out_path} -o {bam_path} --write-index ")

with gzip.open('../data/all_scar_data_wttagged.tsv.gz','wt') as h:
    for library, cell_data in sequences_per_cell.items():
        print(library)
        sam_out_path = f'../data/{library}.sam'
        alignment_commands = []
   
        for cell, observed_sequences in cell_data.items():
            wrote_cell = False
                
            for seq_index, (sequence, n) in enumerate(observed_sequences.most_common()):
                if n<2: # Too little observations
                    continue
                if not wrote_cell:
                    h.write(f'CELL\t{cell}\n')
                    wrote_cell = True
                if sequence in read_alignments_long:
                    scar = ''.join([f'{n}{op}'  for n, op in read_alignments_long[sequence]])
                    is_safe = True
                    is_long = True
                elif sequence in safe_short_to_longer:
                    scar = ''.join([f'{n}{op}' for n, op in read_alignments_short[sequence]])
                    is_safe = True
                    is_long = False
                else:
                    is_long=False
                    if sequence in read_alignments_short:
                        scar = ''.join([f'{n}{op}' for n, op in read_alignments_short[sequence]])
                    else:
                        scar = 'undefined'
                    is_safe = False
                
                h.write(f'\t{n}\t{"Safe" if is_safe else "Amb"}\t{"Long" if is_long else "Short"}\t{scar}\t{sequence}\n')
                # Write to the sam file:

                alignment_commands.append( { # query_name, query_sequence, cell, scar 
                    'query_name': f"{cell}_{seq_index}",
                    "query_sequence": sequence,
                    "cell":cell,
                    "scar":scar
                })
        
        ### Perform the alignments
        align_sequences_to_sam(sam_out_path, alignment_commands)
                    


day7b


  0%|          | 0/159459 [00:00<?, ?it/s]

day4


  0%|          | 0/272335 [00:00<?, ?it/s]

day5c


...

  0%|          | 0/205274 [00:00<?, ?it/s]

day5a


  0%|          | 0/241220 [00:00<?, ?it/s]

day0


  0%|          | 0/227031 [00:00<?, ?it/s]